## Init

In [44]:
import pandas as pd
import sqlite3
import config

def query(sql: str) -> pd.DataFrame:
    """
    Execute a query and yield the results
    """
    conn = sqlite3.connect(config.DB_PATH)
    return pd.read_sql_query(sql, conn)

In [45]:
data = query("SELECT * FROM ledger_items")
data["amount"] = data["amount"].astype(float)
data["month"] = data["tx_date"].str[:7]
data.columns, data['month'].max()

(Index(['tx_id', 'tx_date', 'tx_datetime', 'amount', 'currency', 'description',
        'account', 'ledger_item_type', 'counterparty', 'category', 'labels',
        'month'],
       dtype='object'),
 '2023-02')

## Query

In [46]:
query("SELECT max(tx_date) FROM ledger_items")

,max(tx_date)
0,2023-02-19


## Monthly expenses by category, months

In [47]:
expenses = data[data["ledger_item_type"] == "expense"]
df = expenses.groupby(["month", "category"])["amount"].sum().to_frame()
df = pd.pivot_table(df, index="month", columns="category", values="amount", fill_value=0)
df = df.iloc[::-1]
df.style.background_gradient(cmap='Blues')
# months_by_cat

category,???,Car,Education,Entertainment,Family,Food,Gifts,Healthcare,Hobbies,Home,Loan return,Other,Shopping,Transport,Travel,cdsadsa
month,,,,,,,,,,,,,,,,
2023-02,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-23552.230000
2023-01,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-7488.340000
2022-12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5441.130000
2022-11,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-12257.580000
2022-10,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-11055.850000
2022-09,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-8794.180000
2022-08,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-8280.000000
2022-07,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-7568.920000
2022-06,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9168.850000
